## Imports

In [29]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda, Compose
import matplotlib.pyplot as plt

#### Download training data from open datasets.

In [30]:
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

#### Download test data from open datasets.

In [31]:
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

#### Create data loaders.

In [32]:
batch_size = 64

train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print("Shape of X [N, C, H, W]: ", X.shape)
    print("Shape of y: ", y.shape, y.dtype)
    break

Shape of X [N, C, H, W]:  torch.Size([64, 1, 28, 28])
Shape of y:  torch.Size([64]) torch.int64


#### Get cpu or gpu device for training.

In [33]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using {} device".format(device))

Using cuda device


#### Define model

In [34]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
            nn.ReLU()
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): ReLU()
  )
)


## Optimizing the Model Parameters

In [35]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

def test(dataloader, model):
    size = len(dataloader.dataset)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= size
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model)
print("Done!")

Epoch 1
-------------------------------
loss: 2.302951  [    0/60000]
loss: 2.300988  [ 6400/60000]
loss: 2.287014  [12800/60000]
loss: 2.290298  [19200/60000]
loss: 2.283524  [25600/60000]
loss: 2.246639  [32000/60000]
loss: 2.278758  [38400/60000]
loss: 2.240651  [44800/60000]
loss: 2.247957  [51200/60000]
loss: 2.236766  [57600/60000]
Test Error: 
 Accuracy: 29.1%, Avg loss: 0.035107 

Epoch 2
-------------------------------
loss: 2.228371  [    0/60000]
loss: 2.240587  [ 6400/60000]
loss: 2.186104  [12800/60000]
loss: 2.223596  [19200/60000]
loss: 2.201037  [25600/60000]
loss: 2.120044  [32000/60000]
loss: 2.198223  [38400/60000]
loss: 2.112334  [44800/60000]
loss: 2.147278  [51200/60000]
loss: 2.116405  [57600/60000]
Test Error: 
 Accuracy: 31.2%, Avg loss: 0.033255 

Epoch 3
-------------------------------
loss: 2.111585  [    0/60000]
loss: 2.139847  [ 6400/60000]
loss: 2.019256  [12800/60000]
loss: 2.097026  [19200/60000]
loss: 2.057455  [25600/60000]
loss: 1.917651  [32000/600

#### Saving Models

In [36]:
torch.save(model.state_dict(), "tutorial_model.pth")
print("Saved PyTorch Model State to tutorial_model.pth")

Saved PyTorch Model State to tutorial_model.pth


#### This model can now be used to make predictions

In [38]:
model = NeuralNetwork()
model.load_state_dict(torch.load("tutorial_model.pth"))


classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Ankle boot", Actual: "Ankle boot"
